# GA Implementaiton for predicting a Text

In [ ]:
using Random , Plots
Random.seed!(123)

In [ ]:
const TARGET = "Evolving Ideas with julia"
const TARGET_CHARS = collect(TARGET)
const GENES = vcat(collect('A':'Z'), collect('a':'z'), collect('0':'9'),[' ', '!', ',', '.', '?', '-', ':', ';'])


In [ ]:
random_gene() = rand(GENES)
random_chromosome() = [random_gene() for _ in 1:length(TARGET_CHARS)]

function fitness_finder(genes::Vector{Char})
    error = 0
    for i in eachindex(TARGET_CHARS)
        if genes[i] != TARGET_CHARS[i]
            error += 1
        end
    end
    return error
end

struct Chromosome
    genes::Vector{Char}
    fitness::Int
end

function Chromosome(genes::Vector{Char})
    Chromosome(genes, fitness_finder(genes))
end

function init_population(pop_size::Int)
    [Chromosome(random_chromosome()) for _ in 1:pop_size]
end

In [ ]:
function crossover(parent1::Chromosome, parent2::Chromosome; mutation_rate=0.1)
    child_genes = Char[]
    for i in 1:length(TARGET_CHARS)
        r = rand()
        if r < 0.45
            push!(child_genes, parent1.genes[i])
        elseif r < 1-mutation_rate
            push!(child_genes, parent2.genes[i])
        else
            # Mutation: insert a random gene
            push!(child_genes, random_gene())
        end
    end
    return Chromosome(child_genes)
end

# function crossover(parent1::Chromosome, parent2::Chromosome; mutation_rate=0.5)
#     child_genes = Char[]
#     for i in 1:length(TARGET_CHARS)
#         # Choose gene from parent1 or parent2 with 50% probability each
#         gene = rand() < 0.5 ? parent1.genes[i] : parent2.genes[i]

#         # Apply mutation with given probability
#         if rand() < mutation_rate
#             gene = random_gene()
#         end

#         push!(child_genes, gene)
#     end
#     return Chromosome(child_genes)
# end


In [ ]:
function next_generation(population::Vector{Chromosome})
    # Sort by fitness (best first)
    population_sorted = sort(population, by = c -> c.fitness)
    
    new_generation = Chromosome[]
    
    # 1. Elitism (keep top 15%)
    elite_count = Int(floor(0.15 * length(population)))
    append!(new_generation, population_sorted[1:elite_count])
    
    # 2. Crossover (fill rest of population)
    half = length(population) ÷ 2   # top 50%
    while length(new_generation) < length(population)
        parent1 = rand(population_sorted[1:half])
        parent2 = rand(population_sorted[1:half])
        child = crossover(parent1, parent2)
        push!(new_generation, child)
    end
    
    return new_generation
end



In [ ]:
# Run until perfect match
population = init_population(100)
gen = 0
fitness_history = Int[]

while true
    gen += 1
    population = next_generation(population)
    
    # Find best
    best = findmin([c.fitness for c in population])
    best_chromosome = population[best[2]]
    
    println("Generation $gen | Best candidate: ",
            String(best_chromosome.genes), " | Fitness=", best_chromosome.fitness)
    
    # Save fitness
    push!(fitness_history, best_chromosome.fitness)

    # Stop if perfect
    if best_chromosome.fitness == 0
        println("\n🎉 Target reached in generation $gen")
        break
    end
end

# Plot the convergence
plot(fitness_history, xlabel="Generation", ylabel="Fitness (errors)", 
     title="GA Convergence", legend=false)
